In [6]:
# this doesnt run on colab because you need to have google chrome installed

In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup

In [4]:
# Replace with the appropriate URL, login endpoint, and data labeling page URL
login_url = "https://texera.ics.uci.edu/crowdtagger"
tasks_url = "https://texera.ics.uci.edu/crowdtagger/task/take_task/1197"
# Replace with the appropriate values for the login form


In [5]:
emails = ['wregiste@uci.edu', 'fortinoj@uci.edu', 'kangml@uci.edu', 'kezhang@uci.edu', 'suyeoj2@uci.edu']
dicts = []
for email in emails:
    payload = {
        "username": email,
        "password": "crowdtagger1"
    }
    # Chrome options (headless mode)
    chrome_options = Options()
    chrome_options.add_argument("--headless")

    # Use the path to the ChromeDriver executable on your system
    chromedriver_path = "~/Desktop/chromedriver_mac64/chromedriver"

    # Start a Selenium WebDriver session
    driver = webdriver.Chrome(chromedriver_path, options=chrome_options)

    # Log in
    driver.get(login_url)
    username_field = driver.find_element(By.NAME, "email")
    password_field = driver.find_element(By.NAME, "password")
    username_field.send_keys(payload["username"])
    password_field.send_keys(payload["password"])
    password_field.send_keys(Keys.RETURN)
    # Wait for the login process to complete
    time.sleep(2)
    #Click 'View Current Labeled Results' button
    # Navigate to the data labeling page
    driver.get(tasks_url)
    view_results_button = driver.find_element(By.CSS_SELECTOR, '.btn.btn-default[data-toggle="modal"]')
    view_results_button.click()

    # Wait for the table to load
    time.sleep(2)

    # Set rows per page to maximum
    rows_per_page_dropdown = driver.find_element(By.XPATH, "//div[contains(@class, 'fixed-table-pagination')]//button[contains(@class, 'btn btn-default dropdown-toggle')]")
    rows_per_page_dropdown.click()
    max_rows_option = driver.find_element(By.XPATH, "//ul[@class='dropdown-menu']/li[last()]/a")
    max_rows_option.click()
    time.sleep(2)

    # Extract data from all pages
    data_list = []
    post_counter = 1
    while True:
        # Scrape the table content
        table_html = driver.find_element(By.ID, "curLabelTable").get_attribute("outerHTML")
        soup = BeautifulSoup(table_html, "html.parser")
        table_rows = soup.find_all("tr")

        # Extract the data from the table rows
        empty_rows = 0
        for row in table_rows[1:]:
            columns = row.find_all("td")
            post = post_counter
            post_counter += 1
            labels = columns[1].text

            if labels.strip() == "":
                empty_rows += 1
            else:
                data_list.append((post, labels))

        if empty_rows == len(table_rows[1:]):
            break

        try:
            # Go to the next page
            next_page_button = driver.find_element(By.XPATH, "//a[@class='page-link' and @aria-label='next page']")
            if "disabled" in next_page_button.get_attribute("class"):
                break
            next_page_button.click()
            time.sleep(1)
        except Exception as e:
            print(e)
            break
    driver.quit()
    # Map the data to the appropriate format
    data_dict = {"Post": [item[0] for item in data_list], "Label(s)": [item[1] for item in data_list]}
    dicts.append(data_dict)

<ipython-input-5-42dad5099d3e>:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver_path, options=chrome_options)


WebDriverException: ignored

In [ ]:
data = pd.concat([pd.DataFrame(data_dict) for data_dict in dicts], axis=1)
data.columns = ['0', 'warren_label(s)', 'post_num', 'justin_labels', '1', 'melissa_labels', '2', 'kevin_labels', '3', 'ally_labels']
data.drop(['0', '1', '2', '3'], inplace=True, axis=1)